### Enviroment Initialization


##### Creating Catalog and Schemas

In [0]:
%sql
-- Creating Catalog
CREATE CATALOG IF NOT EXISTS db_Academy
MANAGED LOCATION 'abfss://unity-catalog-storage@dbstoragertkgqhhy66nu6.dfs.core.windows.net/1290381263006333'

In [0]:
%sql
create schema if not exists db_academy.production

In [0]:
%sql
show schemas in db_academy

In [0]:
%sql
describe schema extended db_academy.production

##### Data Source</br>
Volumes has been downloaded from:
[Link to Data Source](https://adb-1290381263006333.13.azuredatabricks.net/marketplace/consumer/listings/ca87a8be-9a17-4469-acee-d992d6cd16ca?o=1290381263006333) </br>
Saved as db_academy_ecomerce under group "Delta Shares Received"

In [0]:
%sql
describe volume db_academy_ecomerce.v01.raw

In [0]:
# exploring Volumes within Catalog
spark.sql("LIST '/Volumes/db_academy_ecomerce/v01/raw/users-historical/'").display()

### Lakeflow Connect

In [0]:
%sql
-- Setting default Catalog & Schema

use catalog db_academy;
use schema production;

SELECT 
  current_catalog(), 
  current_schema()

##### Batch Ingestion

In [0]:
%sql
drop table if exists historical_users_bronze_ctas;

create table historical_users_bronze_ctas as 
select * from read_files(
  '/Volumes/db_academy_ecomerce/v01/raw/users-historical/',
  format => 'parquet'
)


In [0]:
%sql
Select * from db_academy.production.historical_users_bronze_ctas
limit 5

In [0]:
%sql
describe table extended historical_users_bronze_ctas

##### Incremental Ingestion with COPY INTO </br>
- this is legacy option, new way of incremental ingest is to use autoloader

In [0]:
%sql
-- Create Empty table without schema
drop table if exists historical_users_bronze_copy_into;
create table historical_users_bronze_copy_into;

select * from historical_users_bronze_copy_into

In [0]:
%sql
-- Incremental and Idempotent file ingestion
-- Idempotent(means: ingest the same data multiple times and still end up with the same correct final state, without creating duplicates or inconsistencies)
copy into historical_users_bronze_copy_into
from '/Volumes/db_academy_ecomerce/v01/raw/users-historical/'
fileformat = parquet
copy_options ('mergeSchema' = 'true')

In [0]:
%sql
select * From historical_users_bronze_copy_into limit (10)